In [17]:

import sys
import os

sys.path.append('../../../../')
sys.path.append('../../../../python_parser')
sys.path.append('../CodeBLEU')
sys.path.append("../")
retval = os.getcwd()

import json
import logging
import argparse
import warnings
import torch
import time
from utils import Recorder,set_seed
from attacker import Attacker
from transformers import AutoTokenizer, AutoModelForCausalLM,pipeline,RobertaForMaskedLM,RobertaTokenizer
from datasets import load_dataset
from torch.functional import F
from tqdm import tqdm
from CodeBLEU.calc_code_bleu import compute_metrics
from transformers import BitsAndBytesConfig
from peft import PeftModelForCausalLM 
from transformers import AutoTokenizer, AutoModelForCausalLM,pipeline,RobertaForMaskedLM,RobertaTokenizer
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
warnings.simplefilter(action='ignore', category=FutureWarning) # Only report warning\
logger = logging.getLogger(__name__)

additional_special_tokens = {'additional_special_tokens':['<|begin_of_java_code|>','<|end_of_java_code|>'\
                                                           ,'<|begin_of_c-sharp_code|>','<|end_of_c-sharp_code|>',\
                                                            '<|translate|>']}
basemodel = "../finetune/CodeBertForCodeTrans/checkpoint-3220/"
tokenizer = AutoTokenizer.from_pretrained(basemodel,pad_token = "<|pad|>")
model = AutoModelForCausalLM.from_pretrained(basemodel)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device('cpu') 
model.to(device)




Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
If you want to use `RobertaLMHeadModel` as a standalone, add `is_decoder=True.`


In [26]:
def match_c_sharp_code(code:str):
    codelst = code.split('<|begin_of_c-sharp_code|>')
    return ' '.join(codelst[1:])

In [30]:
def get_output_and_Codebleu(code,label):
            
            
            prefix =  additional_special_tokens['additional_special_tokens'][0] 

            input_str = prefix + code + \
                              additional_special_tokens['additional_special_tokens'][1] +\
                additional_special_tokens['additional_special_tokens'][2]
            input = tokenizer(input_str,return_tensors = "pt")
            input.to(device)
            output = model.generate(**input, max_length = 256)
            
            outputs_str = tokenizer.decode(output[0])
            c_sharp_code_of_output = match_c_sharp_code(outputs_str)
            if c_sharp_code_of_output == None or len(c_sharp_code_of_output) == 0:
                return "",0
            output = tokenizer(c_sharp_code_of_output,return_tensors="pt")
            me = compute_metrics((output.input_ids,tokenizer(label, return_tensors="pt").input_ids),tokenizer)
            return c_sharp_code_of_output,me['CodeBLEU']

In [15]:
eval_dataset = load_dataset("json", data_files = "../../data/test_java2cs.jsonl")
eval_dataset = eval_dataset['train']

In [31]:
codebleu = []
for mydict in eval_dataset['translation']:
    javacode = mydict['java']
    c_sharpcode = mydict['cs']
    codebleu.append(get_output_and_Codebleu(c_sharpcode,javacode)[1])
    
    
    

/home/ljc/miniconda3/envs/adv/lib/python3.9/site-packages/transformers/generation/utils.py:1281: UserWarning: Input length of input_ids is 292, but `max_length` is set to 256. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(


/home/ljc/miniconda3/envs/adv/lib/python3.9/site-packages/transformers/generation/utils.py:1281: UserWarning: Input length of input_ids is 370, but `max_length` is set to 256. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(


/home/ljc/miniconda3/envs/adv/lib/python3.9/site-packages/transformers/generation/utils.py:1281: UserWarning: Input length of input_ids is 285, but `max_length` is set to 256. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(


KeyboardInterrupt: 

In [32]:
codebleu

[0.0,
 0.00895947257333783,
 0.0,
 0.0,
 0.12626054947049958,
 0.01716678491365562,
 0.020833333333333332,
 0.002673870777587535,
 0.029339728073155213,
 0.16666666666666666,
 0.024357308922011097,
 0.02992400489714712,
 0.024357308922011097,
 0.02562841510225301,
 0.2714285714285714,
 0.053246298474470766,
 0.0,
 0.036348603276039566,
 0.02138531499473426,
 0.02196421345250725,
 0.036761151121734004,
 0.0002742546961934068,
 0.0,
 0.0417929040999999,
 0.0300837547152772,
 0.0075833629481553335,
 0.029453783469751964,
 0.03302535980923151,
 0.029453783469751964,
 0.08277027027027027,
 0.06495576586394927,
 0.12637228644146153,
 0.029453783469751964,
 0.007602067962509349,
 0.034765830883332696,
 0.0,
 0.0,
 0.029453783469751964,
 0.0,
 0.030396318787700452,
 0.04403696462683085,
 0.12716385209693637,
 0.0,
 0.0,
 0.02,
 0.20900389897312333,
 0.0382748498278164,
 0.1462340084563476,
 0.025039049877286817,
 0.04555232797767736,
 0.07904514235524135,
 0.03661530325400972,
 0.0310327281098